<a href="https://colab.research.google.com/github/rhedouin/Polymetric_landcover/blob/main/lulc_san_francisco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages
!pip install -q earthengine-api geemap


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Change this to your EE Project (find your project name at https://code.earthengine.google.com/)
EE_PROJECT='ee-renaudhedouin'

In [3]:
# Import required libraries
import os, sys
import ee
import geemap
import requests

from datetime import datetime

# Authenticate and initialize the Earth Engine API
try:
    ee.Initialize(project=EE_PROJECT)
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project=EE_PROJECT)

# Define the bounding box coordinates
bounding_box = ee.Geometry.Rectangle([
    -122.6935, 38.1592,  # Lower Left (longitude, latitude)
    -122.2288, 37.4966   # Upper Right (longitude, latitude)
])

# Load Google Earth Engine datasets
# Dynamic World dataset (Sentinel-2 derived)
dynamic_world = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')

# ESA WorldCover dataset (Land Cover)
esa_worldcover = ee.ImageCollection("ESA/WorldCover/v200")

# Define a date range for filtering the Dynamic World dataset
start_date = '2021-01-01'
end_date = '2021-12-31'

# Filter Dynamic World for the bounding box and time range
dw_filtered = dynamic_world.filterBounds(bounding_box).filterDate(start_date, end_date)

# Get the median land cover for Dynamic World
dw_median = dw_filtered.median().clip(bounding_box)

# Get the latest ESA Land Cover image
esa_image = esa_worldcover.filterBounds(bounding_box).first().clip(bounding_box)

# Export Dynamic World median image to local
url_dw = dw_median.getDownloadURL({
    'scale': 10,  # Résolution spatiale en mètres
    'region': bounding_box.getInfo()['coordinates'],
    'crs': 'EPSG:4326',  # Système de coordonnées
    'format': 'GeoTIFF'
})
print('Download URL for Dynamic World:', url_dw)

# Export ESA Land Cover image to local
url_esa = esa_image.getDownloadURL({
    'scale': 10,
    'region': bounding_box.getInfo()['coordinates'],
    'crs': 'EPSG:4326',
    'format': 'GeoTIFF'
})
print('Download URL for ESA Land Cover:', url_esa)

# Télécharger les fichiers localement
# Télécharger Dynamic World
response_dw = requests.get(url_dw)

base_dir = "/home/renaud/Polarimetric_landcover/data"
output_path = os.path.join(base_dir, "Dynamic_world_LULC", "Dynamic_world_LULC.tif")
with open('Dynamic_World_LULC.tif', 'wb') as f:
    f.write(response_dw.content)
print('Dynamic World LULC downloaded successfully.')

# Télécharger ESA Land Cover
output_path = os.path.join(base_dir, "ESA_LandCover", "ESA_LandCover.tif")
response_esa = requests.get(url_esa)
with open('ESA_LandCover.tif', 'wb') as f:
    f.write(response_esa.content)
print('ESA Land Cover downloaded successfully.')


EEException: Total request size (3435173820 bytes) must be less than or equal to 50331648 bytes.